In [3]:
import sys

sys.path.append("../")
sys.path.append("../../")

from difference_matrix import Difference_Matrix

from matrix_sequence import Matrix_Sequence

import numpy as np

from prior_models import normal_prior

### General Performance tests for matrix inversion algorithms

Let's create a second order difference matrix

In [4]:
D=Difference_Matrix(n=100,k=1)

In [5]:
D.D

array([[ 1., -2.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1., -2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -2.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1., -2.,  1.]])

In [6]:
D.compute_k_difference(0)

array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [7]:
D.D.shape

(98, 100)

### Large computations of Difference Matrix


In [8]:
D=Difference_Matrix(1000,1)

In [9]:
D=Difference_Matrix(10,2)
D.D

array([[ 1., -3.,  3., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1., -3.,  3., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1., -3.,  3., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1., -3.,  3., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -3.,  3., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -3.,  3., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -3.,  3., -1.]])

### High order difference matrix

In [10]:
D=Difference_Matrix(100,3)
D.D

array([[ 1., -4.,  6., ...,  0.,  0.,  0.],
       [ 0.,  1., -4., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -4.,  1.,  0.],
       [ 0.,  0.,  0., ...,  6., -4.,  1.]])

### Confirm Difference Matrix Recursion works

In [11]:
D=Difference_Matrix(2500,1)
d_21=D.compute_k_difference(21)
d_21

array([[  1., -22., 231., ...,   0.,   0.,   0.],
       [  0.,   1., -22., ...,   0.,   0.,   0.],
       [  0.,   0.,   1., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   1.,   0.,   0.],
       [  0.,   0.,   0., ..., -22.,   1.,   0.],
       [  0.,   0.,   0., ..., 231., -22.,   1.]])

In [12]:
D=Difference_Matrix(2500,21)


 WARNING Condition number is large: 8.874296950555368e+17


In [13]:
assert (np.allclose(d_21,D.D))

### Time Difference Matrix Computation

In [14]:
D=Difference_Matrix(100,1)

T=Difference_Matrix(100,k=1,t=np.linspace(0,1,100))

In [15]:
D=Difference_Matrix(100,2)



In [16]:
D=Difference_Matrix(100,1)

T=Difference_Matrix(100,k=1,t=np.linspace(0,1,100))

In [17]:
D.DDT

array([[ 6., -4.,  1., ...,  0.,  0.,  0.],
       [-4.,  6., -4., ...,  0.,  0.,  0.],
       [ 1., -4.,  6., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  6., -4.,  1.],
       [ 0.,  0.,  0., ..., -4.,  6., -4.],
       [ 0.,  0.,  0., ...,  1., -4.,  6.]])

### Edge Cases

In [16]:
D=Difference_Matrix(100,0)



Condition number of D is 63.65674116287194


In [17]:
T=Difference_Matrix(100,k=0,t=np.linspace(0,1,100))

Condition number of D is 63.65674116287194
Condition number of D is 63.65674116287194


In [18]:
D=Difference_Matrix(100,0)
D.D

Condition number of D is 63.65674116287194


array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [19]:
truncated_D=Difference_Matrix(99,0)
truncated_D.D


Condition number of D is 63.02006849910265


array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [20]:
truncated_D.D.dot(D.D)

array([[ 1., -2.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1., -2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -2.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1., -2.,  1.]])